In [ ]:
from cryptography.fernet import Fernet
import json

#### User control

In [ ]:
def read_write_json(data: dict = None, filename="credentials.json") -> dict:
    if data is None:
        with open(filename, 'r') as f:
            data = json.load(f)
    else:
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)
    return data

In [ ]:
def create_user(login, password):
  data = read_write_json()
  if login in data:
    return "User already exists."
  else:
    data[login] = password
    read_write_json(data)
  return "User created successfully."


In [ ]:
def user_login(login, password):
  data = read_write_json()
  if not login in data:
    return "User does not exists."
  if data[login] != password:
    return "Wrong password."
  return True

#### Cryptography

In [ ]:
def generate_key():
  return Fernet.generate_key()

def encrypt(message: bytes, key: bytes) -> bytes:
    return Fernet(key).encrypt(message)

def decrypt(token: bytes, key: bytes) -> bytes:
    return Fernet(key).decrypt(token)

In [ ]:
def decrypt_middleware(message: str, key: bytes):
  decrypt_msg = decrypt(message, key)
  return json.loads(decrypt_msg.decode())

def encrypt_middleware(message: dict, key: bytes):
  message_str = json.dumps(message)
  return encrypt(message_str.encode(), key)

In [ ]:
# client
def client_example(message: dict):
  # criar usuario "command": "create user login password"
  msg_string = json.dumps(message)
  key = generate_key()
  msg_enc = encrypt(msg_string.encode(), key)
  return msg_enc, key

# server
def server_example(client_message, key):
  result = decrypt_middleware(client_message, key)
  # check create login
  if result["command"].startswith('create user'):
    splitted_cmd = result["command"].split()
    login = splitted_cmd[2]
    password = splitted_cmd[3]
    create = create_user(login, password)
    return encrypt_middleware({'result': create}, key)

  login = user_login(result['login'], result['password'])
  if login != True:
    return encrypt_middleware({'result': login}, key)
  command = result['command']
  # executa comando passando 'state' e 'ledname', salva em variavel 'exec_result'
  exec_result = 'exemplo'
  return encrypt_middleware({'result': exec_result}, key)

In [ ]:
client_message, key = client_example({"login":"elton", "password":"senha", "command": "RED 1"})
server_example(client_message, key)

b'gAAAAABkiLrpm_xvveOXwB50JhDquV-gaoaYerfv3DaTXYWkTs2BbMw8CMiTHhz3Gn50z5W34XtrVWpxSkXJkmz2ndb58OYWqiZq4ctrNHufFMMtmBTYbChQn4whYlS8Z7M8A_Z7S2h-'